### Imports

In [2]:
import os
import pandas as pd
import numpy as np
from functools import reduce


In [3]:
data_dir = "../base_data"
new_data_dir = "new_data"

In [4]:
columns_to_keep_teams = {
    'teamRoster.csv': ['teamId', 'teamName'],
    'standings.csv': ['teamId', 'form', 'teamRank', 'teamId','gamesPlayed','wins', 'ties', 'losses', 'points','gf','ga','gd'],
}


In [5]:
dfs_teams = []

for filename, columns in columns_to_keep_teams.items():
    path = os.path.join(data_dir, filename)
    df = pd.read_csv(path, usecols=columns)
    dfs_teams.append(df)

In [6]:
merged_df_teams = reduce(lambda left, right: pd.merge(left, right, on='teamId', how='inner'), dfs_teams)

# Generate ordered column list automatically
ordered_columns_t = []
for cols in columns_to_keep_teams.values():
    for col in cols:
        if col not in ordered_columns_t:
            ordered_columns_t.append(col)

# Reorder columns safely
merged_df_teams = merged_df_teams[[col for col in ordered_columns_t if col in merged_df_teams.columns]]
merged_df_teams = merged_df_teams.dropna()


In [ ]:
cols_to_int = ['teamRank', 'gamesPlayed', 'wins', 'ties', 'losses', 'points', 'gf', 'ga', 'gd']
merged_df_teams[cols_to_int] = merged_df_teams[cols_to_int].astype(int)


In [10]:
merged_df_teams.head()

,teamId,teamName,form,teamRank,gamesPlayed,wins,ties,losses,points,gf,ga,gd
0,21361,Kyoto Sanga,WDDLD,14,38,12,11,15,47,43,55,-12
1,21361,Kyoto Sanga,WLLWD,3,17,8,4,5,28,24,21,3
2,21361,Kyoto Sanga,WDDLD,14,38,12,11,15,47,43,55,-12
3,21361,Kyoto Sanga,WLLWD,3,17,8,4,5,28,24,21,3
4,21361,Kyoto Sanga,WDDLD,14,38,12,11,15,47,43,55,-12


In [8]:
# df = merged_df_teams.groupby(['eventId']).agg(
# homeTeamId=pd.NamedAgg(column="homeTeamId", aggfunc="first"),
# awayTeamId=pd.NamedAgg(column="awayTeamId", aggfunc="last"),
# homeTeamScore=pd.NamedAgg(column="homeTeamScore", aggfunc="first"),
# awayTeamScore=pd.NamedAgg(column="awayTeamScore", aggfunc="last"),
# homeTeamWinner=pd.NamedAgg(column="homeTeamWinner", aggfunc="first"),
# awayTeamWinner=pd.NamedAgg(column="awayTeamWinner", aggfunc="last"),
# possessionPct_home=pd.NamedAgg(column="possessionPct", aggfunc="first"),
# possessionPct_away=pd.NamedAgg(column="possessionPct", aggfunc="last"),
# totalShots_home=pd.NamedAgg(column="totalShots", aggfunc="first"),
# totalShots_away=pd.NamedAgg(column="totalShots", aggfunc="last"),
# shotsOnTarget_home=pd.NamedAgg(column="shotsOnTarget", aggfunc="first"),
# shotsOnTarget_away=pd.NamedAgg(column="shotsOnTarget", aggfunc="last"),
# saves_home=pd.NamedAgg(column="saves", aggfunc="first"),
# saves_away=pd.NamedAgg(column="saves", aggfunc="last"),
# wonCorners_home=pd.NamedAgg(column="wonCorners", aggfunc="first"),
# wonCorners_away=pd.NamedAgg(column="wonCorners", aggfunc="last"),
# ).reset_index()

In [9]:
output_path_teams = os.path.join(new_data_dir, 'database_teams.csv')

os.makedirs(os.path.dirname(output_path_teams), exist_ok=True)

merged_df_teams.to_csv(output_path_teams, index=False)

print(f"Merged CSV saved to {output_path_teams}")


Merged CSV saved to new_data\database_teams.csv
